# Principal Component Analysis (PCA\)

## Descrição do algoritmo

Principal Component Analysis (PCA) é um método de redução de dimensionalidade que se utiliza da identificação de combinações lineares para descrever atributos de dados e projetá-los em espaços menores [[1]](https://www.ibm.com/docs/en/db2-warehouse?topic=procedures-principal-component-analysis-pca).

O algortimo, em resumo, projeta a "coordenadas" de pontos que representam dados em eixos ortogonais que acentuam as diferenças entre suas posições. Esse eixos são conhecidos como Componentes Principais (do inglês "Principal Components" e de onde vem o nome da técnica). Com esse processo, é construído um novo sistema de representação de dados baseado nos componentes principais, que são capazes de representar melhor os dados com um número menor de váriaveis [[2]](https://www.ibm.com/topics/principal-component-analysis)[[3]](https://youtu.be/FgakZw6K1QQ?si=FgdL70D16oavWbHg).

## Proposta de uso

Tendo em vista que o PCA atua reduzindo a complexidade da informação de uma base de dados com a redução no número de váriaveis, ele apresenta grande potencial de uso no projeto ao permitir que tornemos os dados disponíveis em equivalentes mais compreensíveis.

A piori, espera-se utilizar o algoritmo para plotar gráficos mais compreensíevis de modo a permitir melhor entendimento de como a informação está disponível e definir qual o tipo de problema que estamos enfrentando para, então, que possamos escolher os melhores modelos e algoritmos para enfrentá-lo.


## Aplicação

Estão descritos todos os passos executados para a realização de análises usando o método de PCA, bem como a interpretação dos resultados. Cada um dos subtópicos exploram uma tabela de dados distinta.

O seguinte trecho realiza a importação das bibliotecas necessárias para o funcionamento do algoritmo:

In [ ]:
# Importa bibliotecas necessárias para o funcionamento do algoritmo
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px
from sklearn import decomposition

### Análise times série A

A primeira análise a ser realizada será com a tabela referente aos times da série A do Campeonato Brasileiro. Essa base de dados contém diversas informações referentes ao desempenho de diferentes clubes ao longo do torneio, apresentando, por exemplo, número de gols no decorrer da temporada e quantidade de cartões recebidos por jogadores do time.

Espera-se, com essa análise, compreender como os dados estão estruturados e se há algum tipo de relação entre eles.

Abertura do arquivo:

In [ ]:
dfTimesSerieA = pd.read_csv('/content/drive/Shareddrives/Drive IBMatch/Data/brazil-serie-a-teams2-2024-to-2024-stats.csv') # Abre o arquivo
dfTimesSerieA = dfTimesSerieA.drop(columns=dfTimesSerieA.select_dtypes('object')) # Elimina colunas com dados de texto

Definição de quantos componentes principais serão gerados e criação dos novos dados com atributos pertencentes ao PCA:

In [ ]:
pcaTSA = decomposition.PCA(n_components = 5) # Determina quantos componentes principais serão gerados
pcaTSA_features = pcaTSA.fit_transform(dfTimesSerieA) # Aplica o algoritmo de PCA e salva os novos dados em função dos componentes principais
pcaTSA.explained_variance_ratio_

#### Gráficos e análises preliminares

O trecho a seguir gera o gráfico Scree para a análise anterior. Um gráfico Scree descreve quanto cada componente (eixo) é relevante no posicionamento dos pontos no espaço gerado a partir da aplicação do PCA por meio da quantidade de variância nas variáveis originais [[4]](https://sanchitamangale12.medium.com/scree-plot-733ed72c8608) [[5]](https://www.ibm.com/docs/pt-br/spss-statistics/saas?topic=reduction-total-variance-explained). Na prática, é equivalente a dizer quanto cada atributo é relevante na diferenciação de determinada informação.

In [ ]:
# Plota gráfico Scree
fig = px.bar(x=range(1, len(pcaTSA.explained_variance_ratio_)+1),
             y=pcaTSA.explained_variance_ratio_,
             title='Gráfico Scree (Relação de Percentual de Variância e Componente Principal)',
             labels={'x':'Componente Principal', 'y':'Percentual de variância'})
fig.show()

A partir da observação do gráfico anterior, é interessante notar que os dois primeiros componentes são responsáveis por quase 100% de toda a variação de posição de pontos no gráfico gerado a partir dos dados e que têm como eixo os próprios componentes principais.

Sendo assim, no contexto de utilização dos dados para realização das análises, é aceitável a utilização desses componentes com maior impacto em detrimento daqueles menos relevantes, visto a diferença na ordem de grandeza que cada um deles possui.

Dessa maneira, também observa-se a efetividade do método PCA na redução de dimensionalidade, no qual, após a sua aplicação, tornou-se possível representar as informações da tabela em um gráfico, como se segue:

In [ ]:
# Plota gráfico bidimensional com os dados gerados a partir do método de PCA

# Determina os eixos X e Y
x = pcaTSA_features[:, 0]
y = pcaTSA_features[:, 1]

# Plota gráfico
chart = px.scatter(x=x, y=y,
                   title="Relação entre Componentes Principais 1 e 2",
                   labels={'x':'Componente Principal 1', 'y':'Componente Principal 2'})
chart.show()

Observa-se, por meio da disposição dos dados no gráfico acima, como estes estão dispostos de maneira quase linear. Nesse sentido, é perceptível que conforme os valores do eixo do "Componente principal 1", eixo x, crescem, crescem também o valores no eixo do "Componente principal 2", eixo y, mas somentes se esse valor for maior que 0. Com isso, se estabelece uma relação de proporcionalidade direta (conforme um valor aumenta, ocorre um aumento no outro valor).

Entretanto, quando analisamos valores do "Componente principal 2" menores que 0, percebemos que os valores se alinham de modo a formar uma diagonal descendente, indicando que há uma relação inversalmenet proporcional entre os valores, na qual, conforme os valores do eixo x crescem, os do eixo y diminuem.

A presença desse relacionamento linear nos dados pode ser um indicativo de que a utilização de métodos regressivos, usados para determinar tendências, possa ser efetivo em processos de predição.


Voltand ao gráfico Scree debatido anteriormente, constatou-se a presença de 3 componentes principais gerados a partir dos dados. Apesar dos dois primeiros componentes possuirem uma relevancia muito maior na diferenciação dos pontos, foi gerado um gráfico para representação tridimensional dos pontos, a fim de compreender como as informações se comportam quando dispostas em mais dimensões, como apresentado no gráfico a seguir:

In [ ]:
# Plota gráfico tridimensional dos pontos
fig = px.scatter_3d(pcaTSA_features,
                    x= pcaTSA_features[:, 0],
                    y=pcaTSA_features[:, 1],
                    z=pcaTSA_features[:,2],
                    title='Relação entre componentes principais 1, 2 e 3',
                    labels={'x':'Componente Principal 1', 'y':'Componente Principal 2', 'z':'Componente Principal 3'})
fig.show()

Apesar de, a primeira vista, parecer que os pontos estão tridimensionalmente dispostos de maneira a formar três linhas que partem de um mesmo ponto e segem em direções distintas de modo equivalente, como um tripé fotográfico, esse efeito é meramente resultado da escala aplicada a cada um dos eixos. Caso seja realizada uma leitura mais atenta, percebe-se que enquanto o eixo X possui uma escala que varia de -20 mil unidades e segue até 50 mil unidades e o eixo Y uma escala que varia de -40 mil unidades e segue até 40 mil unidades, o eixo Z possui uma escala que varia apenas de -4 mil unidades até 8 mil unidades, tendo uma ordem de grandeza consideravelmente inferior às unidades anteriores.

Nesse sentido, caso o gráfico fosse representado com todos os eixos contendo a mesma medida, teríamos uma figura bem mais planificada no eixo Z, se assemelhando mais, assim, à figura obtida do segundo gráfico desta seção.

#### Considerações gerais

A partir das análises gráficas realizadas anteriormente, nota-se que os dados dessa base apresentam uma tendência de seguir algum tipo de relação linear, sendo diretamente proporcional caso os valores sejam maiores que um valor desconhecido e próximo a 0 e inversamente proporcionais no caso de valores abaixo desse limite. Essa relação linear pode ser um indicativo de que seja vantajosa a utilização de modelos de regressão linear para a predição de resultados futuros, sendo necessárias mais análises para confirmação de existência dessas relações.

### Análise partidas da série A

Seguindo com as análises, esta seção tem o objetivo de explorar a tabela com dados relativos as partidas, que já tenham ocorrido ou não, da série A do Campeonato Brasileiro. Essa base de dados contém diversas informações sobre as partidas, apresentando, por exemplo, quantidade de gols e chutes a gol.

Espera-se, com essa análise, compreender como os dados estão estruturados e se há algum tipo de relação entre eles.

Abertura do arquivo:

In [ ]:
dfPartidasSerieA = pd.read_csv('/content/drive/Shareddrives/Drive IBMatch/Data/brazil-serie-a-matches-2024-to-2024-stats_edit.csv', sep=";") # Abre arquivo
dfPartidasSerieA = dfPartidasSerieA.drop(columns=dfPartidasSerieA.select_dtypes(include=['object'])).fillna(0) # Elimina colunas com dados de texto e preenche espaços vazios com o número 0

Definição de quantos componentes principais serão gerados e criação dos novos dados com atributos pertencentes ao PCA:

In [ ]:
pcaPSA = decomposition.PCA(n_components = 5) # Determina quantos componentes principais serão gerados
pcaPSA_features = pcaPSA.fit_transform(dfPartidasSerieA) # Aplica o algoritmo de PCA e salva os novos dados em função dos componentes principais
pcaPSA.explained_variance_ratio_ # Apresenta

#### Gráficos e análises preliminares

O trecho a seguir gera o gráfico Scree para a análise anterior. Um gráfico Scree descreve quanto cada componente (eixo) é relevante no posicionamento dos pontos no espaço gerado a partir da aplicação do PCA [[4]](https://sanchitamangale12.medium.com/scree-plot-733ed72c8608). Na prática, é equivalente a dizer quanto cada atributo é relevante na diferenciação de determinada informação.

In [ ]:
# Plota gráfico Scree
fig = px.bar(x=range(1, len(pcaPSA.explained_variance_ratio_)+1),
             y=pcaPSA.explained_variance_ratio_,
             title='Gráfico Scree (Relação de Percentual de Variância e Componente Principal)',
             labels={'x':'Componente Principal', 'y':'Percentual de variância'})
fig.show()

A partir da observação do gráfico anterior, nota-se como o Componente Principal 1 representa um percentual de variância que representa, sozinho, quase 100%. Esse resultado indica que, para a aplicação do método PCA na base de dados anterior unicamente da maneira em que ela está, a redução de dimensionalidade está gerando apenas 1 componente principal com uma relenvância demasiadamente alta.  

Com o objetivo de entender como essa redução influencia na disposição dos dados, foi realizada a produção de um gráfico bidimensional relacionando os Componentes Principais 1 e 2:

In [ ]:
# Plota gráfico bidimensional com os dados gerados a partir do método de PCA

# Determina os eixos X e Y
x = pcaPSA_features[:, 0] # Componente Principal 1
y = pcaPSA_features[:, 1] # Componente Principal 2

# Plota gráfico
chart = px.scatter(x=x, y=y,
                   title="Relação entre Componentes Principais 1 e 2",
                   labels={'x':'Componente Principal 1', 'y':'Componente Principal 2'})
chart.show()

Por meio de uma breve análise do gráfico anterior, percebe-se que grande parte dos dados estão concentrados na parte inferior do gráfico, com valores do Componente Principal 2 variando entre -2 mil e mil, que representa uma variação muito pequena dentro do contexto do gráfico.

Essa concentração de valores apresenta ainda um comportamento bastante liner, formando uma diagonal quase perfeita e bem distribuida ao longo do eixo dos valores de Componente Principal 1. Ademais, também se destaca no gráfico a presença de 9 pontos que fogem da diagonal citada anteriormente.

A configuração peculir os dados assumiram quando distribuídos no gráfico e a disparidade do percentual de variância atribuido a cada componente principal levantaram dúvidas acerca do real posicionamento dos dados. A fim de averiguar o comportamento dos dados em gráficos com dimensões superiores e averiguar se há algum padrão de distribuição, construiu-se um gráfico que combina os valores dos 3 primeiros componentes principais formando eixos de um espaço de 3 dimensões com o valor do Componente Principal 4, representado pela variação da cor dos pontos no gráfico:

In [ ]:
# Plota gráfico tridimensional dos pontos
fig = px.scatter_3d(pcaPSA_features,
                    title="Relação entre Componentes Principais 1, 2, 3 e 4",
                    labels={'x':'Componente Principal 1', 'y':'Componente Principal 2', 'z':'Componente Principal 3', 'color':'Componente Principal 4'},
                    x= pcaPSA_features[:, 0],
                    y=pcaPSA_features[:, 1],
                    z=pcaPSA_features[:,2],
                    color=pcaPSA_features[:,3])
fig.show()

A análise do gráfico anterior revela diversos fatores osbre o comportamento dos dados nesta tabela. A priori, observa-se a grande concentração de dados em um plano paralelo ao plano formado pelos eixos do Componente Principal 1 e Componente Principal 3, sendo esse plano uma outra visão da diagonal formada no gráfico anterior.

Olhando essa concentração de dados lançando maior atenção sobre o valor do Componente Principal 4, e consequentemente a cor dos pontos, nota-se que pontos próximos tendem a ter coloração semelhante, salvo casos específicos, de modo a formar um gradiente com a variação dos valores.

Por fim, uma última constatação possível se refere a escala dos eixos, as quais diferem muito entre si. Nesse sentido, cabe destacar a diferença na ordem de grandeza da escala dos Componentes Principais, pois, enquanto os valores do Componentes Principais 2, 3 e 4 variam em dezenas de centenas, dezenas de -100 a 100 e dezenas de -60 a 60, respectivamente, o Componente Principal 1 tem uma variação em seu eixo da ordem dos milhões, o que explica sua dominação no percentual de variância.


#### Análise secundária

As observações levantadas nos blocos anteriores trouxeram questionamentos acerca do impacto que determinadas colunas desempenham no resultado do PCA e consequentemente na forma como os dados são dispostos nos gráficos.

Nesse sentido, foi realizada uma segunda exploração na tabela, na qual foram foram identificadas tabelas que acrescentavam informações de baixa relevância às previsões, como o horário de início do jogo em Unix Time, semana da partida e o número de espectadores no jogo.

A partir dessa segunda exploração, iniciou-se uma segunda análise na mesma tabela, mas agora eliminando esses valores, como exposto a seguir:


In [ ]:
dfPartidasSerieA2 = pd.read_csv('/content/drive/Shareddrives/Drive IBMatch/Data/brazil-serie-a-matches-2024-to-2024-stats_edit.csv', sep=";") # Abre o arquivo da base de dados
temp = dfPartidasSerieA2 # Gera variável temporária para pareamento de valores (uso mais adiante na análise)
dfPartidasSerieA2 = dfPartidasSerieA2.drop(columns=dfPartidasSerieA2.select_dtypes(include=['object'])).fillna(0) # Remove colunas com dados de texto e preenche espaços vazios com o número 0
dfPartidasSerieA2 = dfPartidasSerieA2.drop(columns=['timestamp', 'attendance', 'Game Week']) # Remove as colunas com horário do jogo em unix time, número de espectadores e a semana do jogo

Definição de quantos componentes principais serão gerados e criação dos novos dados com atributos pertencentes ao PCA:





In [ ]:
pcaPSA2 = decomposition.PCA(n_components=5)
pcaPSA2_features = pcaPSA2.fit_transform(dfPartidasSerieA2)
pcaPSA2.explained_variance_ratio_

Gera gráfico Scree para a análise anterior. Um gráfico Scree descreve quanto cada componente (eixo) é relevante no posicionamento dos pontos no espaço gerado a partir da aplicação do PCA [[4]](https://sanchitamangale12.medium.com/scree-plot-733ed72c8608). Na prática, é equivalente a dizer quanto cada atributo é relevante na diferenciação de determinada informação.

In [ ]:
# Plota gráfico Scree
fig = px.bar(x=range(1, len(pcaPSA2.explained_variance_ratio_)+1),
             y=pcaPSA2.explained_variance_ratio_,
             title='Gráfico Scree (Relação de Percentual de Variância e Componente Principal)',
             labels={'x':'Componente Principal', 'y':'Percentual de variância'})
fig.show()

É notável a diferença entre o Gráfico Scree acima com aquele gerado antes da remoção das colunas númericas de menor relevância. Agora, o Percentual de Variância não está mais concentrado no priemiro componente principal, mas distribuída em todos os componentes principais gerados, representando que todos têm uma participação considerável na diferenciação de elementos.

Percebida essa diferença, partiu-se para a geração de gráficos para a comparação dos resultados e tentativa de identificação de novos padrões nos dados.

Como primeira tentativa, foi gerado um gráfico de dispersão que relaciona os Componentes Principais 1 e 2, com utilização de cores para diferenciação dos status dos jogos. Com "status" entende-se o estado atual do jogo no sentido de ocorrência, tendo como valores possíveis "complete", "suspended" e "incomplete" (completo, suspenso e incompleto, respectivamente). O objetivo dessa análise é constatar se há alguma relação de proximidade entre os dados com mesmo status.

In [ ]:
# Plota gráfico bidimensional com os dados gerados a partir do método de PCA

frame = pd.concat([temp['status'], pd.DataFrame(pcaPSA2_features)], axis=1) # "Etiqueta" elementos para elaboração de um gráfico com diferenciação de status de jogo

# Determina os eixos X e Y
x = frame[0]
y = frame[1]

# Plota o gráfico
chart = px.scatter(x=x, y=y,
                   color=frame['status'],
                   title="Relação entre Componentes Principais 1 e 2 com Status do Jogo",
                   labels={'x':'Componente Principal 1', 'y':'Componente Principal 2', 'color':'Status do jogo'})
chart.show()


Assim como no Gráfico Scree exposto anteriormente, a dispersão dos dados e a maneira como os valores dos componentes principais estão relacionados no gráfico acima em muito diferem dos resultados alcançados na primeira análise.

Partindo para uma análise do gráfico, percebe-se a formação de dois aglomerdos distintos, um relativo aos jogos que já ocorreram e outro com jogos que não ocorreram, seja por estarem suspensos ou por ainda não terem acontencido. Também é possível observar que os dados nesses aglomerados estão distribuídos ao longo de duas diagonais paralelas expondo uma relação inversamente proporcional entre os componentes principais 1 e 2. Ademais, destaca-se também uma concentração de pontos de jogos completos que está distante do aglomerado principal, apesar de também seguirem, de algum modo, a diagonal formada por esses elementos.

Com o intuito de averiguar se há algum tipo de padrão de distribuição dos dados dos jogos que ocorreram realizou-se a remoção do aglomerado de elementos dos jogos que não ocorreram e a coloração dos elementos com base no resultado da partida (Empate, time da casa vencedor ou time visitante vencedor).

Segue o trecho de código e o gráfico resultante:

In [ ]:

# "Etiqueta" os dados das partidas completas, dividindo-os em vitórias da casa, do visitante e empate
winners = []
temp = temp[temp['status'] == 'complete']

for j, i in temp.iterrows():
  if i['home_team_goal_count'] > i['away_team_goal_count']:
    winners.append('Casa')
  elif i['home_team_goal_count'] < i['away_team_goal_count']:
    winners.append('Visitante')
  else:
    winners.append('Empate')

temp['status'] = winners # Atribui os resultados das partidas ao status do elemento

# Junta "etiquetas" com os elementos do PCA
frame = pd.concat([temp['status'], pd.DataFrame(pcaPSA2_features)], axis=1)

# Determina os eixos X e Y
x = frame[0]
y = frame[1]

# Configura o gráfico
chart = px.scatter(x=x, y=y,
                   color=frame['status'],
                   title="Relação entre Componentes Principais 1 e 2 com Vencedor do Jogo",
                   labels={'x':'Componente Principal 1', 'y':'Componente Principal 2', 'color':'Vencedor do Jogo'})

# Apresenta o gráfico
chart.show()

Por meio desse gráfico é possível enxergar como estão distribuídos os resultados das partidas ao longo do aglomerado dos jogos que já ocorreram. Percebe-se que os elementos estão muito bem distribuídos ao longo do aglomerado, sem apresentar uma relação clara em 2 dimensões.

Mais testes envolvendo tentativas de previsão e exploração de dimensões superiores devem ser feitos para constatar a presença (ou ausência) de relação entre esses dados.

Buscando, também, averiguar se há algum tipo de relacionamento entre os dados quando olhamos para sua disposição em mais dimensões, construiu-se um gráfico semelhante a àquele apresentado na subseção anterior, no qual relaciona-se os Componentes Principais 1, 2 e 3, tornando-os eixos em um gráfico de dispersão tridimensional, com o Componente Principal 4, representando a cor dos elementos dentro de uma faixa determinada, como se segue:

In [ ]:
# Configura gráfico tridimensional dos pontos
fig = px.scatter_3d(pcaPSA2_features,
                    x=pcaPSA2_features[:, 0],
                    y=pcaPSA2_features[:, 1],
                    z=pcaPSA2_features[:,2],
                    color=pcaPSA2_features[:,3],
                    title='Relação entre Componentes Principais 1, 2, 3 e 4',
                    labels={'x':'Componente Principal 1','y':'Componente Principal 2','z':'Componente Principal 3','color':'Componente Principal 4'})

# Apresenta gráfico
fig.show()

Ao analisar o gráfico acima e compará-lo ao gráfico gerado com a base antes da remoção das colunas citadas no início desta subseção, torna-se perceptível quão diferente são os resultado obtidos no que tange a distribuição e entendimento dos elementos.

Em primeira análise, cabe destacar a escala dos eixos dos componentes principais, todas na mesma ordem de grandeza, juntamente com a escala de coloração. Tal observação é reafirmada quando se considera o gráfico Scree gerado anteriormente, o qual já revelava um grau de importância semelhante entre os componentes principais.

Ademais, outro ponto de destaque é a configuração que os diversos elementos assumiram quando dispostos no espaço, formando dois planos paralelos a partir dos aglomerados de pontos, indicando mais uma vez a correlação que existe entre eles.

Um ponto de atenção no gráfico são os agrupamentos de cores dos pontos. Apesar da formação dos aglomerados, nota-se como a distribuição das cores, e consequentemente dos valores do Componente Principal 4, ocorreu de maneira maneira distinta nos dois _clusters_. Em um deles, naquele com valores de Componente Principal 2 menores que 0 (que agrupa os elementos relativos as partidas que não ocorreram), as cores se detiveram a valores semelhantes, causando certa uniformidade de coloração quando o todo é analisado em conjunto. Já no outro aglomerado, observa-se que as cores possuem grande variação, abrangendo quase todo o espectro disponível (com valores extremos inclusos).

Esse arranjo na distribuição das cores expõe como as relações entre o posicionamento e as características individuais de cada elemento se desenvolvem de maneira ainda mais complexa, mesmo com os procedimentos para redução de dimensionalidade usados no método de PCA. A compreensão mais aprofundada desses relacionamentos é um interessante ponto de partida para análises posteriores, sendo necessários mais experimentos e testes para explorar a forma como tais fatores interagem para alcançar determinado resultado.

#### Considerações gerais

A partir das análises realizadas gráficas realizadas anteriormente, nota-se as complexas relações existentes entre os atributos de cada elemento. Apesar dessa complexidade, é perceptível como os elementos dessa base possuem relacionamentos definidos que podem ser explorados em futuras análise e experimentações. Também é interessante citar a riqueza dessa base para utilização de modelos preditivos, podendo ser aplicada uma variade de técnicas para a realização de diferentes previsões do comportamento dos dados.

Por fim, destaca-se também como a efetividade do uso da PCA nessa base, bem como a importância da limpeza e pré-processamento de dados para a geração de resultados aproveitáveis para outras análises.